# Aproach 1 uploading the data from csv files without combining the data

In [1]:
from elasticsearch import Elasticsearch 
import os
import pandas as pd
import numpy as np

In [2]:
es = Elasticsearch([{'host':'localhost','port':9200,'scheme':'http'}])

In [3]:
es.ping()

True

In [5]:
index_name = 'genre'

In [18]:
def upload_csvs(csv_file):
    df = pd.read_csv(csv_file)
    df.replace({np.nan: None}, inplace=True)
    data = df.to_dict(orient = 'records')
    for doc in data:
        es.index(index= index_name, body=doc)
    es.indices.refresh(index = index_name)
  

In [19]:
folder = '/media/anirudh/Volume G/Githu/archive (3)/'

In [20]:
for filename in os.listdir(folder):
    if filename.endswith('.csv'):
        csv_file_p = os.path.join(folder,filename)
        upload_csvs(csv_file_p)
        print(f"Uploaded {filename} to Elasticsearch index {index_name}")
print('all files are uploaded')

Uploaded animation.csv to Elasticsearch index genre
Uploaded romance.csv to Elasticsearch index genre
Uploaded action.csv to Elasticsearch index genre
Uploaded crime.csv to Elasticsearch index genre
Uploaded mystery.csv to Elasticsearch index genre
Uploaded sports.csv to Elasticsearch index genre
Uploaded biography.csv to Elasticsearch index genre
Uploaded history.csv to Elasticsearch index genre
Uploaded fantasy.csv to Elasticsearch index genre
Uploaded film-noir.csv to Elasticsearch index genre
Uploaded thriller.csv to Elasticsearch index genre
Uploaded adventure.csv to Elasticsearch index genre
Uploaded horror.csv to Elasticsearch index genre
Uploaded scifi.csv to Elasticsearch index genre
Uploaded family.csv to Elasticsearch index genre
Uploaded war.csv to Elasticsearch index genre
all files are uploaded


In [44]:
# Define the search query
search_query = {
    "query": {
        "multi_match": {
            "query": "demon slayer",    
            "type": "cross_fields",        
            "fields": ["movie_name", "genre"]         }
    },
    'size': 25
}
response = es.search(index=index_name, body=search_query)
unique_movie_names = set()
for hit in response['hits']['hits']:
    movie_name = hit['_source']['movie_name']
    genre = hit['_source']['genre']
    director = hit['_source'].get('director', 'N/A')
    year = hit['_source'].get('year', 'N/A')
    runtime = hit['_source'].get('runtime','N/A')
    if movie_name not in unique_movie_names:
        print(f"Movie Name: {movie_name}")
        print(f"Runtime: {runtime}")
        print(f"Genre: {genre}")
        print(f"Director: {director}")
        print(f"Year: {year}")
        print("\n")
        unique_movie_names.add(movie_name)

Movie Name: Immortal Demon Slayer
Runtime: 123 min
Genre: Action, Adventure, Family
Director: Chi-Kin Kwok
Year: 2017


Movie Name: The Last Demon Slayer
Runtime: None
Genre: Action, Adventure, Fantasy
Director: Shaky González
Year: 2011


Movie Name: Slayer
Runtime: None
Genre: Fantasy
Director: None
Year: I


Movie Name: Demon Slayer the Movie: Mugen Train
Runtime: 117 min
Genre: Animation, Action, Adventure
Director: Haruo Sotozaki
Year: 2020


Movie Name: Demon Slayer: Kimetsu no Yaiba - Asakusa Arc
Runtime: 103 min
Genre: Animation, Action, Fantasy
Director: Haruo Sotozaki
Year: 2021


Movie Name: Demon Slayer: Kimetsu no Yaiba - Sibling's Bond
Runtime: 105 min
Genre: Animation, Action, Fantasy
Director: Haruo Sotozaki
Year: 2019


Movie Name: Sin Slayer
Runtime: None
Genre: Action
Director: Eisenstein
Year: None


Movie Name: The Slayer
Runtime: 90 min
Genre: Horror, Mystery, Thriller
Director: J.S. Cardone
Year: 1982


Movie Name: Dragon Slayer
Runtime: None
Genre: Fantasy
Direc

# Aproach 2 combining the data from all the csv files and then upload to Elasticsearch 

In [7]:
import os
import pandas as pd

folder_path = '/media/anirudh/Volume G/Githu/archive (3)/'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

combined_data = pd.concat([pd.read_csv(os.path.join(folder_path, file)) for file in csv_files], ignore_index=True)
print(combined_data)


          movie_id                                         movie_name  year  \
0        tt3915174                       Puss in Boots: The Last Wish  2022   
1        tt6718170                        The Super Mario Bros. Movie  2023   
2       tt26537229  Demon Slayer: Kimetsu No Yaiba - To the Swords...  2023   
3        tt1488589                     Guillermo del Toro's Pinocchio  2022   
4       tt14668630                              Lyle, Lyle, Crocodile  2022   
...            ...                                                ...   ...   
736595  tt26698087                                          Janissary   NaN   
736596  tt26690183                                World War 1 project   NaN   
736597  tt26695367                                                 74   NaN   
736598  tt26698193                                     Pandora Palace  2019   
736599  tt26733697                   Spion 11: Operasjon Traelandsfos  2001   

       certificate  runtime                        

In [3]:
combined_csv_path = '/media/anirudh/Volume G/Githu/archive (3)/combined.csv'

combined_data.to_csv(combined_csv_path, index=False)


In [8]:
df = combined_data.to_dict(orient = 'records')

In [9]:
df

[{'movie_id': 'tt3915174',
  'movie_name': 'Puss in Boots: The Last Wish',
  'year': '2022',
  'certificate': 'PG',
  'runtime': '102 min',
  'genre': 'Animation, Adventure, Comedy',
  'rating': 7.9,
  'description': 'When Puss in Boots discovers that his passion for adventure has taken its toll and he has burned through eight of his nine lives, he launches an epic journey to restore them by finding the mythical Last Wish.',
  'director': 'Joel Crawford, \nJanuel Mercado',
  'director_id': '/name/nm3150455/',
  'star': 'Antonio Banderas, \nSalma Hayek, \nHarvey Guillén, \nFlorence Pugh',
  'star_id': '/name/nm2591093/,/name/nm0000104/,/name/nm0000161/,/name/nm2957490/,/name/nm6073955/',
  'votes': 93143.0,
  'gross(in $)': 168464485.0},
 {'movie_id': 'tt6718170',
  'movie_name': 'The Super Mario Bros. Movie',
  'year': '2023',
  'certificate': 'PG',
  'runtime': '92 min',
  'genre': 'Animation, Adventure, Comedy',
  'rating': nan,
  'description': 'The story of The Super Mario Bros. on

In [17]:
id_map = {
    "settings": {
        "number_of_shards": 1,  
        "number_of_replicas": 1  
    },
    "mappings": {
        "properties": {
            "movie_name": {"type": "text"},
            "year": {"type": "integer","ignore_malformed" : True},
            "certificate": {"type": "keyword", "ignore_malformed" : True},
            "runtime": {"type": "keyword"},
            "genre": {"type": "text"},
            "rating": {"type": "float"},
            "description": {"type": "text"},
            "director": {"type": "text"},
            "director_id": {"type": "keyword"},
            "star": {"type": "text"},
            "star_id": {"type": "keyword"},
            "votes": {"type": "integer", "ignore_malformed" : True},
            "gross(in $)": {"type": "float", "ignore_malformed" : True}
        }
    }
}


In [18]:
index_names = 'movie_&_tvshows_data'



In [19]:
if es.indices.exists(index=index_names):
    print(f"The index '{index_names}' exists.")

else:
    print(f"The index '{index_names}' does not exist.")
    es.indices.create(index=index_names, body=id_map, ignore=400)   

The index 'movie_&_tvshows_data' does not exist.


/tmp/ipykernel_9636/360139974.py:6: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_names, body=id_map, ignore=400)


In [20]:
for movie in df:
    for key, value in movie.items():
        if pd.isna(value):
            movie[key] = None
    es.index(index=index_names, body=movie)
print('upload_sucess')

upload_sucess


In [21]:
query = {
    'query':{
        'match_all':{}
    }
}
es.search(index=index_names,body= query)['hits']['hits']

[{'_index': 'movie_&_tvshows_data',
  '_id': 'x7gOH44BZK1i7zg12zqW',
  '_score': 1.0,
  '_source': {'movie_id': 'tt0055060',
   'movie_name': 'Kultainen vasikka',
   'year': '1961',
   'certificate': None,
   'runtime': '91 min',
   'genre': 'Comedy, Drama, Romance',
   'rating': 6.2,
   'description': 'Add a Plot',
   'director': 'Ritva Arvelo',
   'director_id': '/name/nm0038095/',
   'star': 'Aino Mantsas, \nHelge Herala, \nMarja Korhonen, \nToivo Mäkelä',
   'star_id': '/name/nm0544189/,/name/nm0378427/,/name/nm0466255/,/name/nm0617429/',
   'votes': 82.0,
   'gross(in $)': None}},
 {'_index': 'movie_&_tvshows_data',
  '_id': 'yLgOH44BZK1i7zg12zqd',
  '_score': 1.0,
  '_source': {'movie_id': 'tt0022686',
   'movie_name': 'The Man Who Won',
   'year': '1932',
   'certificate': None,
   'runtime': '87 min',
   'genre': 'Comedy, Drama, Romance',
   'rating': None,
   'description': 'Add a Plot',
   'director': 'Norman Walker',
   'director_id': '/name/nm0908076/',
   'star': 'Henry Ke

In [35]:
query = {
    "query": {
        "multi_match": {
            "query": "animation",    
            "type": "cross_fields",        
            "fields": ["movie_name", "genre"]  
        }
    },
    'size': 25
}

response = es.search(index='movie_&_tvshows_data', body=query)

unique_movie_names = set()

for hit in response['hits']['hits']:
    movie_name = hit['_source']['movie_name']
    if movie_name not in unique_movie_names:
        print(movie_name)
        unique_movie_names.add(movie_name)

Bremen Musicians
Águila y Jaguar: Los Guerreros Legendarios
Bible Town
When You Get to the Forest
Nasreddin Hoca Zaman Yolcusu
Krishna - The Birth
Contra-Filé
Where's the Dragon?
Speed Racer: The Movie
The Toy Warrior
100 Years of Disney Animation: A Shorts Celebration
Chhota Bheem and the ShiNobi Secret
Natsume's Book of Friends: The Waking Rock and the Strange Visitor
Kaka, kevad ja teised
The Immortal Warrior
Jurassic Bark
The Spider and the Fly: The Movie
Only a Dream
Sea Monsters
Memoir of a Snail
Mighty Raju Rio Calling
Bheem vs Aliens
Washingtonia
New Initial D the Movie: Legend 2 - Racer
Tom Gates Is Fantastic at Some Things


In [39]:
query = {
    "query": {
        "multi_match": {
            "query": "animation comedy",   
            "type": "cross_fields",        
            "fields": ["movie_name", "genre"]  
    },
    'size': 25
}

response = es.search(index='movie_&_tvshows_data', body=query)
unique_movie_names = set()
for hit in response['hits']['hits']:
    movie_name = hit['_source']['movie_name']
    genre = hit['_source']['genre']
    director = hit['_source'].get('director', 'N/A')
    year = hit['_source'].get('year', 'N/A')
    runtime = hit['_source'].get('runtime','N/A')
    if movie_name not in unique_movie_names:
        unique_movie_names.add(movie_name)


Movie Name: Memoirs of an Imaginary Friend
Runtime: None
Genre: Animation, Comedy
Director: Sacha Baron Cohen
Year: None


Movie Name: Immigrants (L.A. Dolce Vita)
Runtime: 78 min
Genre: Animation, Comedy
Director: Gabor Csupo
Year: 2008


Movie Name: The Old Man: The Movie
Runtime: 88 min
Genre: Animation, Comedy
Director: Oskar Lehemaa, 
Mikk Mägi
Year: 2019


Movie Name: Jokes My Folks Never Told Me
Runtime: 82 min
Genre: Animation, Comedy
Director: Gerry Woolery
Year: 1978


Movie Name: Footrot Flats: The Dog's Tale
Runtime: 71 min
Genre: Animation, Comedy
Director: Murray Ball
Year: 1986


Movie Name: Boonie Bears: A Mystical Winter
Runtime: 96 min
Genre: Animation, Comedy
Director: Leon Ding, 
Fuyuan Liu
Year: 2015


Movie Name: The Waterman Movie
Runtime: None
Genre: Animation, Comedy
Director: Bryan Waterman
Year: None


Movie Name: Little Johnny: The Movie
Runtime: 78 min
Genre: Animation, Comedy
Director: Ralph Moser
Year: 2011


Movie Name: Koty Ermitazha
Runtime: 70 min
Gen

# Search Inference with tkinter

In [13]:
import tkinter as tk
from elasticsearch import Elasticsearch

class MovieSearchApp:
    def __init__(self, master):
        self.master = master
        master.title("Movie Search System")

        self.label = tk.Label(master, text="Search for a movie:")
        self.label.pack()

        self.search_entry = tk.Entry(master, width=50)
        self.search_entry.pack()

        self.search_button = tk.Button(master, text="Search", command=self.search_movies)
        self.search_button.pack()

        self.results_text = tk.Text(master, wrap=tk.WORD, width=50, height=10)
        self.results_text.pack()

        self.es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme':'http'}])
        self.index_name = 'movie_&_tvshows_data'

    def search_movies(self):
        query = self.search_entry.get()
        search_query = {
            "query": {
                "multi_match": {
                    "query": query,
                    "type": "cross_fields",
                    "fields": ["movie_name", "genre"]
                }
            },
            'size': 25
        }
        response = self.es.search(index=self.index_name, body=search_query)
        hits = response['hits']['hits']  
        if hits:
            self.display_results(hits)
        else:
            self.results_text.insert(tk.END, "No results found.\n")

    def display_results(self, hits):
        self.results_text.delete('1.0', tk.END)
        for hit in hits:
            movie_name = hit['_source']['movie_name']
            genre = hit['_source']['genre']
            runtime = hit['_source'].get('runtime', 'N/A')
            self.results_text.insert(tk.END, f"Movie: {movie_name}\nGenre: {genre}\n\nDuration: {runtime}\n\n")

def main():
    root = tk.Tk()
    app = MovieSearchApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
